In [1]:
import numpy as np
import tensorflow as tf

In [8]:
# Assuming TensorFlow 2.x
# Define `eland0` and `deland` as constants
eland0 = tf.constant(5.9, dtype=tf.float32)  # Example initial value
deland = tf.constant(0.1, dtype=tf.float32)   # Example decay rate

# Create a sequence of time steps from 1 to 81 (inclusive), representing 5 years per period
# This means we'll generate values [1, 6, 11, ..., 396] representing the years in 5-year intervals
t = tf.range(1, 82, dtype=tf.float32)

# Compute `eland(t)` for each time step
eland_t = eland0 * (1 - deland) ** (t - 5)  # Adjusting the exponent to start at 0 (1st period)

# Print the result
print(eland_t[0])

tf.Tensor(8.992533, shape=(), dtype=float32)
